In [32]:
!pip install uv
!uv pip install -r requirements.txt


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Audited 9 packages in 14ms


# Prerequisite

## Ollama
Ollama platform is used to run the LLM and embedding models. See [doc](https://github.com/ollama/ollama) for setting ollama locally.

## pgvector
Open-source vector similarity search for Postgres. This is used to store the documents and the embeddings. See [doc](https://github.com/pgvector/pgvector) for setting up the database.

# Variables

Export the following variables in your environment:

* PGVector database variables 
  * VECTORDB_HOST
  * VECTORDB_USERNAME
  * VECTORDB_PASSWORD
  * VECTORDB_PORT
  * VECTORDB_DBNAME

* Embedding function variables
  * EMBEDDING_FUNCTION - Model to be used as the embedding function. Defaults to llama3.1
  * EMBEDDING_FUNCTION_URL - Base url of the model. Defaults to localhost:11434

* LLM variable
  * LLM_MODEL - Model to be used for answering the questions. Defaults to llama3.
  * LLM_MODEL_URL - Base url of the model. Defaults to localhost:11434

Alternatively, you can set your variables in a `.env` file.


In [33]:
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

DB_HOST = os.getenv("VECTORDB_HOST", "localhost")
DB_PORT = os.getenv("VECTORDB_PORT", "5432")
DB_USERNAME = os.getenv("VECTORDB_USERNAME", "postgres")
DB_PASSWORD = os.getenv("VECTORDB_PASSWORD", "")
DB_NAME = os.getenv("VECTORDB_DBNAME", "postgres")

EMBEDDING_FUNCTION = os.getenv("EMBEDDING_FUNCTION", "llama3.1")
EMBEDDING_FUNCTION_URL = os.getenv("EMBEDDING_FUNCTION_URL", "localhost:11434")

LLM_MODEL = os.getenv("LLM_MODEL", "llama3.1")
LLM_MODEL_URL = os.getenv("LLM_MODEL_URL", "localhost:11434")

# Dataset

In [34]:
import pandas as pd

data = pd.read_csv("spill_notes.csv", index_col=False)
data.reset_index(drop=True, inplace=True)
data.head()

,id,service_id,service_type,rate,hour,amount,description,report_no,ip_address_identifier,owner,...,type,projected_eta_temp_2,actual_eta_temp_2,legacy_id,excavation_time,projected_eta,actual_eta,incident_no,state_incident_no,work_start_time
0,935602,36,Receipt of Assignment From Client,85,0.5,42.5,Initial information-gathering conversation wit...,NaN,NaN,NaN,...,fixed,NaN,NaN,0,NaN,0,0,NaN,NaN,NaN
1,935604,63,Locate Contractor Services For Recovery Operat...,25,0.0,25.0,Spills Management Org identifies EnviroServe a...,NaN,NaN,NaN,...,fixed,NaN,NaN,0,NaN,0,0,NaN,NaN,2024-09-04 23:41:25
2,935607,17,Dispatch Contractor For Corrective Actions Bas...,75,0.0,75.0,Rosa Behn with R&L calls and reports a release...,NaN,NaN,NaN,...,fixed,NaN,NaN,0,NaN,120,0,NaN,NaN,2024-09-04 23:44:22
3,935608,65,Response Authorization,85,0.0,0.0,The contractor provides an ETA of 2 hours. T...,NaN,NaN,NaN,...,hourly,NaN,NaN,0,NaN,0,0,NaN,NaN,2024-09-04 23:49:09
4,935610,34,Regulatory Notification To Local Agency,75,0.0,75.0,Spills Management Org had called and left a vo...,NaN,NaN,NaN,...,fixed,NaN,NaN,0,NaN,0,0,NaN,NaN,2024-09-04 23:56:47


# Embedding and Vector Database

A Postgres database with `pgvector` extension is used to store the documents and the embeddings.

In [35]:
import json

import tiktoken
from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import DataFrameLoader
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
from langchain_postgres import PGVector


CONNECTION_STRING = (
    f"postgresql+psycopg://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)
COLLECTION_NAME = "spill_notes"


def num_tokens_from_string(string: str, encoding_name="cl100k_base") -> int:
    if not string:
        return 0
    # Returns the number of tokens in a text string
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def convert_table_to_docs(data: pd.DataFrame) -> pd.DataFrame:
    rec = []
    data_json = data.to_dict(orient='records')
    for i, r in data.iterrows():
        rec.append([r["id"], json.dumps(data_json[int(i)])])
    
    preprocessed_data = pd.DataFrame(rec, columns=["id", "content"])
    chunked_data = chunk_data(preprocessed_data, 'content')
    loader = DataFrameLoader(chunked_data, page_content_column="content")
    return loader.load()


def get_embedding(embedding_model: str, url: str):
    if embedding_model in ("llama3.1"): 
        return OllamaEmbeddings(model=embedding_model, base_url=url)

    raise NotImplementedError(f"{embedding_model} is not supported")


def insert_docs_to_vectordb(
    docs: list[Document], collection_name: str, connection, embedding
):
    vector_store = PGVector(
        embeddings=embedding,
        collection_name=collection_name,
        connection=connection,
        use_jsonb=True,
    )
    vector_store.add_documents(docs, ids=[doc.metadata["id"] for doc in docs])


def chunk_data(
    data: pd.DataFrame, target_column: str, chuck_size: int = 512
) -> pd.DataFrame:
    text_splitter = TokenTextSplitter(chunk_size=chuck_size, chunk_overlap=103)

    new_list = []
    for i in range(len(data.index)):
        text = data[target_column][i]
        token_len = num_tokens_from_string(text)
        if token_len <= chuck_size:
            r = []
            for c in data.columns:
                r.append(data[c][i])
            new_list.append(r)
        else:
            # split text into chunks using text splitter
            split_text = text_splitter.split_text(text)
            for j in range(len(split_text)):
                r = []
                for c in data.columns:
                    d = split_text[j] if c == target_column else data[c][i]
                    r.append(d)
                new_list.append(d)

    return pd.DataFrame(new_list, columns=data.columns)

In [36]:
docs = convert_table_to_docs(data)
embedding = get_embedding(EMBEDDING_FUNCTION, EMBEDDING_FUNCTION_URL)
insert_docs_to_vectordb(
    docs, COLLECTION_NAME, CONNECTION_STRING, embedding
)

# RAG pipeline

In [37]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama


def get_documents(vector_store: PGVector, query: str) -> str:
    docs = vector_store.similarity_search(query, k=10)
    return "\n--\n".join([doc.page_content for doc in docs])


def get_llm(model: str, url: str):
    if model in ('llama3.1'):
        return ChatOllama(model=model, base_url=url, num_ctx=8000)
    raise NotImplementedError(f"{model} is not supported")


vector_store = PGVector(
    embeddings=get_embedding("llama3.1", "localhost:11434"),
    collection_name=COLLECTION_NAME,
    connection=CONNECTION_STRING,
    use_jsonb=True,
)

prompt = PromptTemplate(
    template="""You are a helpful assistant that helps in answering questions based on the provided context below,
    enclosed within <context></context> XML tags.
    <context>
    {context}
    </context>

    When answering the user:
    - If you don't know the answer, simply state that you don't know.
    - If you're unsure, seek clarification.
    - Avoid mentioning that the information was sourced from the context.
    - Respond in accordance with the language of the user's question and provide the context.
    
    Question: {question}
    Answer:
    """,
    input_variables=["question", "context"]
)

model = get_llm(LLM_MODEL, LLM_MODEL_URL)

rag_chain = prompt | model | StrOutputParser()

query = "what is the GPS coordinates provided by the contractor for service_id 19?"
docs = get_documents(vector_store, query)
print(f"Question: {query}")
answer = rag_chain.invoke({"question": query, "context": docs})
print(f"Answer: {answer}\n")

query = "What is service_type for service_id 19?"
docs = get_documents(vector_store, query)
print(f"Question: {query}")
answer = rag_chain.invoke({"question": query, "context": docs})
print(f"Answer: {answer}\n")

Question: what is the GPS coordinates provided by the contractor for service_id 19?
Answer: The contractor provides GPS coordinates, which are 41.026944, -82.088056.

Question: What is service_type for service_id 19?
Answer: Project Review With Contractor Services.

